# Proyecto: Procesamiento de Datos con Python 2023 
## API del Sistema de Información Económica (SIE) de Banco de México

## Solicitud de prueba a la API, análisis de la estructura de los datos obtenidos y conversión de JSON a Data Frame


### Solicitud de prueba a la API

En esta sección se relizará una petición para probar como funciona la API del proyecto.

Para usar la [SIE-API](https://www.banxico.org.mx/SieAPIRest/service/v1/) de Banco de México es necesario:
* Token, se obtiene [aquí](https://www.banxico.org.mx/SieAPIRest/service/v1/token)
* Endpoint o url, ubicado en la [documentación](https://www.banxico.org.mx/SieAPIRest/service/v1/doc/consultaDatosSeries) correspondiente a datos históricos
* Idenidentificador de serie de tiempo, se consulta en el catálogo de [catálogo de series](https://www.banxico.org.mx/SieAPIRest/service/v1/doc/catalogoSeries)

In [92]:
# Se importan las bibliotecas requests y pandas para realizar llamadas HTTP y manipular datos respectivamente. 
import requests
import pandas as pd

In [93]:
# Token obtenido en el sitio SIE API de Banco de México. 
token = '5d4442d9cd23918613a61626f0dacf97bba0afd996fa0b923c6657b6eeb26297'

# URL para hacer consultas a la API de Banco de México.
endpoint = 'https://www.banxico.org.mx/SieAPIRest/service/v1/series/'

# Definición de un diccionario para almacenar los parametros de la funcion get de la libreria requests. 
params = {
    "token" : token
}

In [94]:
# Definición de las listas que almacenan los identificadores de series a consultar.
# Cómo se menciona el reporte escrito, la API permite consultas en grupos de máximo 20 identificadores.
# Cada identificador de serie corresponde a una columna del data frame final.  
id_cuentas_validas = ['SF335591-SF335610/datos', 'SF335611-SF335629/datos', 'SF339757/datos', 'SF350480/datos']

In [95]:
# Se realiza una solicitud GET para obtener del API una respuesta con los parametros definidos. 
r = requests.get(endpoint+id_cuentas_validas[0], params = params)

In [96]:
# Se verifica el código que indica el estado de la respuesta a la solicitud HTTP realizada a la API. 
r.status_code

200

### Análisis de la estructura de los datos obtenidos

Una vez que se verificó que la solicud se realizó correctamente, se analiza la estructura de los datos obtenidos. 

El análisis incluye: 
* Verificar el tipo de objeto de la respuesta (lista, diccionario, tupla, etc.)
* Explorar la estructura de los datos para definir la manera más adecuada de extraerlos. 

En este caso se encontró que los datos se estructuraban en varias listas anidadas con diccionarios.

In [97]:
# Se obtiene la respuesta codificada con el método json()
r.json()

# Se observa que la respuesta obtenida se trata de un objeto tipo diccionario. 
type(r.json())

dict

In [98]:
# Se obtienen las claves del diccionario, en este caso sólo existe la clave 'bmx'.  
r.json().keys()

dict_keys(['bmx'])

In [99]:
# Se observa el tipo de objeto que contiene la clave 'bmx', es un diccionario.  
type(r.json()['bmx'])

dict

In [100]:
# Se obtienene las claves del primer diccionario anidado, sólo existe la clave 'series'  
r.json()['bmx'].keys()

dict_keys(['series'])

In [101]:
# Se observa el tipo de objeto que contiene la clave 'series', es una lista.
type(r.json()['bmx']['series'])

list

In [102]:
# Se analiza el primer elemento de la lista que contiene la clave 'series'.
# Se determina que es una lista de diccionarios.   
r.json()['bmx']['series'][0]

{'idSerie': 'SF335595',
 'titulo': 'Cuentas validadas de Opciones Empresariales del Noreste, S.A. de C.V.',
 'datos': [{'fecha': '24/05/2020', 'dato': '5'},
  {'fecha': '25/05/2020', 'dato': '2'},
  {'fecha': '26/05/2020', 'dato': '2'},
  {'fecha': '27/05/2020', 'dato': '1'},
  {'fecha': '28/05/2020', 'dato': '1'},
  {'fecha': '29/05/2020', 'dato': '0'},
  {'fecha': '30/05/2020', 'dato': '0'},
  {'fecha': '31/05/2020', 'dato': '0'},
  {'fecha': '01/06/2020', 'dato': '2'},
  {'fecha': '02/06/2020', 'dato': '3'},
  {'fecha': '03/06/2020', 'dato': '0'},
  {'fecha': '04/06/2020', 'dato': '2'},
  {'fecha': '05/06/2020', 'dato': '1'},
  {'fecha': '06/06/2020', 'dato': '0'},
  {'fecha': '07/06/2020', 'dato': '0'},
  {'fecha': '08/06/2020', 'dato': '1'},
  {'fecha': '09/06/2020', 'dato': '0'},
  {'fecha': '10/06/2020', 'dato': '0'},
  {'fecha': '11/06/2020', 'dato': '2'},
  {'fecha': '12/06/2020', 'dato': '0'},
  {'fecha': '13/06/2020', 'dato': '0'},
  {'fecha': '14/06/2020', 'dato': '0'},
  {

In [103]:
# Por comodidad, se asigna la lista de diccionarios a la variable prueba_lista
prueba_lista = r.json()['bmx']['series']

In [104]:
# Se analiza la lista de diccionarios, en este caso se trata de una lista de 20 
# elementos que corresponden a cada una de las series solicitadas en la prueba al API. 
len(prueba_lista)

20

In [105]:
# Se accede al primer diccionario de la lista para analizarlo a detalle. 
# Se trata de un diccionario con las claves: 'idSerie', 'titulo' y 'datos'.
print(type(prueba_lista[0]))
print(prueba_lista[0].keys())

<class 'dict'>
dict_keys(['idSerie', 'titulo', 'datos'])


In [106]:
# Se accede a las claves 'idSerie' y 'titulo'. 
# Estas son cadenas que contienen el identificador de la serie y el título de la entidad financiera. 
print(prueba_lista[0]['idSerie'])
print(type(prueba_lista[0]['idSerie']))
print(prueba_lista[0]['titulo'])
print(type(prueba_lista[0]['titulo']))

SF335595
<class 'str'>
Cuentas validadas de Opciones Empresariales del Noreste, S.A. de C.V.
<class 'str'>


In [107]:
# Se accede a la clave 'datos' que es una lista de diccionarios con las fechas y datos deseados.
print(type(prueba_lista[0]['datos']))
print(prueba_lista[0]['datos'])

<class 'list'>
[{'fecha': '24/05/2020', 'dato': '5'}, {'fecha': '25/05/2020', 'dato': '2'}, {'fecha': '26/05/2020', 'dato': '2'}, {'fecha': '27/05/2020', 'dato': '1'}, {'fecha': '28/05/2020', 'dato': '1'}, {'fecha': '29/05/2020', 'dato': '0'}, {'fecha': '30/05/2020', 'dato': '0'}, {'fecha': '31/05/2020', 'dato': '0'}, {'fecha': '01/06/2020', 'dato': '2'}, {'fecha': '02/06/2020', 'dato': '3'}, {'fecha': '03/06/2020', 'dato': '0'}, {'fecha': '04/06/2020', 'dato': '2'}, {'fecha': '05/06/2020', 'dato': '1'}, {'fecha': '06/06/2020', 'dato': '0'}, {'fecha': '07/06/2020', 'dato': '0'}, {'fecha': '08/06/2020', 'dato': '1'}, {'fecha': '09/06/2020', 'dato': '0'}, {'fecha': '10/06/2020', 'dato': '0'}, {'fecha': '11/06/2020', 'dato': '2'}, {'fecha': '12/06/2020', 'dato': '0'}, {'fecha': '13/06/2020', 'dato': '0'}, {'fecha': '14/06/2020', 'dato': '0'}, {'fecha': '15/06/2020', 'dato': '0'}, {'fecha': '16/06/2020', 'dato': '0'}, {'fecha': '17/06/2020', 'dato': '0'}, {'fecha': '18/06/2020', 'dato': '1

In [112]:
# Se analiza la clave 'datos' y se observa que nuevamente se trata de una lista de diccionarios. 
print(type(prueba_lista[0]['datos'][0]))
prueba_lista[0]['datos'][0]

<class 'dict'>


{'fecha': '24/05/2020', 'dato': '5'}

### Prueba conversión de JSON a Data Frame
Una vez que se identificó la estructura de los datos, se definió la forma de extraerlos y convertirlos en un data frame. Para esto fue necesario: 

* Usar un `for loop` que recorra la lista `prueba_lista`
* Convertir cada elemento de la lista en un data frame con `pd.DataFrame()`
* Usar el valor de la clave `'fecha'` como índice del data frame
* Convertir los valores del indice a un tipo de dato `datetime64`
* Usar el valor de la clave `'titulo'` como nombre de la columna
* Definir la lista `lista_provisional` para guardar cada data frame procesado con el método `append()`

Después de tener la lista de dataframes completa, se unieron cada uno de estos
con el método `pd.contact()` y se indicó el parametro `axis=1` para hacer la concatenación horizontalmente. 

El resultado fue un data frame de 982 filas y 20 columnas, las columnas corresponden al primer grupo de identificadores de series que se usaron para la solicitud HTTP de prueba hacia la API. 

In [116]:
lista_provisional = []
for i in prueba_lista:
    df_interno = pd.DataFrame(i['datos'])
    df_interno = df_interno.set_index('fecha')
    df_interno.index = pd.to_datetime(df_interno.index, format='%d/%m/%Y')
    df_interno.rename(columns={'dato': i['titulo']}, inplace=True)
    lista_provisional.append(df_interno)

df_prueba = pd.concat(lista_provisional, axis=1)
df_prueba

,"Cuentas validadas de Opciones Empresariales del Noreste, S.A. de C.V.","Cuentas validadas de Banco Ahorro Famsa, S.A.","Cuentas validadas de Bansi, S.A.","Cuentas validadas de Banco Bancrea, S.A.",Total de cuentas validadas,"Cuentas validadas de ABC Capital, S.A.","Cuentas validadas de Banco Base, S.A.",Cuentas validadas de Banco Regional S.A.,"Cuentas validadas de Banco Autofin México, S.A.","Cuentas validadas de Banco Mercantil del Norte, S.A.","Cuentas validadas de BanCoppel, S.A.","Cuentas validadas de BBVA Bancomer, S.A.","Cuentas validadas de Banco del Ahorro Nacional y Servicios Financieros, S.N.C.","Cuentas validadas de Banco Actinver, S.A.","Cuentas validadas de Banca Afirme, S.A.","Cuentas validadas de Banco Azteca, S.A.","Cuentas validadas de Banco Nacional del Ejército, Fuerza Aérea y Armada","Cuentas validadas de Banco del Bajío, S.A.","Cuentas validadas de Banco Nacional de México, S.A.","Cuentas validadas de Bankaool, S.A."
fecha,,,,,,,,,,,,,,,,,,,,
2020-05-24,5,12,0,0,"20,477",0,1,362,2,339,"2,571","12,572",0,0,41,718,20,21,"1,352",0
2020-05-25,2,13,0,1,"9,422",0,0,141,1,161,942,"5,689",0,0,11,294,9,9,502,0
2020-05-26,2,12,0,0,"9,494",0,1,158,2,135,910,"5,652",1,0,27,243,6,4,624,0
2020-05-27,1,11,0,0,"11,850",0,0,121,0,119,800,"8,761",0,0,16,233,10,9,565,0
2020-05-28,1,28,0,0,"13,902",0,0,140,1,175,"1,003","10,299",0,0,16,297,18,15,603,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-26,6,N/E,0,0,"11,476",0,0,124,3,295,842,"7,913",0,0,24,375,5,14,"1,177",0
2023-01-27,17,N/E,0,0,"12,159",0,0,124,0,352,857,"8,602",0,1,37,449,11,26,965,0
2023-01-28,1,N/E,0,0,"9,989",0,0,78,1,282,888,"6,835",1,0,18,391,10,19,835,0


## Automatización de solicitudes a la API y transformación de JSON a Data Frame 

### Automatización de solicitudes a la API

Este punto se realizó con un `for loop` para recorrer cada elemento de la lista `id_cuentas_validas` que almacena los identificadores de las series de tiempo en rangos de 20. 

Se usó un bloque `try-except` para manejar posibles errores o excepciones en el código. 

Además, solo si el código del estado de la respuesta es `200`, se ejecuta el bloque siguiente: 
* Se asigna la lista de diccionarios a la variable `lista_datos`
* Se asignan todas las listas del ciclo en turno a la variable `lista_cuentas_validas` con el método `.extend()`

En caso de que el código del estado de la respuesta no sea `200`, se ejecutará un mensaje avisando del error. 

In [120]:
lista_cuentas_validas = []
for id in id_cuentas_validas:
  try:
    r = requests.get(endpoint+id, params = params)
    if r.status_code == 200:
      lista_datos = r.json()['bmx']['series']
      lista_cuentas_validas.extend(lista_datos)
    else: print(f"Ha ocurrido un error: {r.status_code}")       
  except: 
    continue

In [ ]:
lista_cuentas_validas

### Automatización de transformación de JSON a Data Frame

Se usa el mismo código que el de la sección **Prueba conversión de JSON a Data Frame** pero con la lista que se obtuvo en el punto anterior: `lista_cuentas_validas`. 

El resultado final es el data frame `df_cuentas_validas`, igual al que se uso durante el proyecto de 982 filas y 41 columnas.

Finalmente se guarda el data frame obtenido en un archivo `.csv` con el método `to_csv()`

In [125]:
lista_cocat = []
for i in lista_cuentas_validas:
    df_interno = pd.DataFrame(i['datos'])
    df_interno = df_interno.set_index('fecha')
    df_interno.index = pd.to_datetime(df_interno.index, format='%d/%m/%Y')
    df_interno.rename(columns={'dato': i['titulo']}, inplace=True)
    lista_cocat.append(df_interno)

df_cuentas_validas = pd.concat(lista_cocat, axis=1)
df_cuentas_validas

,"Cuentas validadas de BBVA Bancomer, S.A.","Cuentas validadas de Banco Bancrea, S.A.",Cuentas validadas de Banco Regional S.A.,"Cuentas validadas de Banco del Ahorro Nacional y Servicios Financieros, S.N.C.","Cuentas validadas de Banco Actinver, S.A.","Cuentas validadas de Banco Azteca, S.A.","Cuentas validadas de BanCoppel, S.A.","Cuentas validadas de Banco Base, S.A.","Cuentas validadas de Bankaool, S.A.","Cuentas validadas de Banco del Bajío, S.A.",...,"Cuentas validadas de Banco Invex, S.A.","Cuentas validadas de Sistema de Transferencias y Pagos STP, S.A. de C.V.","Cuentas validadas de Banco Inbursa, S.A.","Cuentas validadas de Banco Multiva, S.A.","Cuentas validadas de Banco Inmobiliario Mexicano, S.A.","Cuentas validadas de Consubanco, S.A","Cuentas validadas de Banco Sabadell, S.A.","Cuentas validadas de Fundación Dondé Banco, S.A.","Cuentas validadas de Accendo Banco, S.A.","Cuentas validadas de Operadora de Pagos Móviles de México, S.A de C.V."
fecha,,,,,,,,,,,,,,,,,,,,,
2020-05-24,"12,572",0,362,0,0,718,"2,571",1,0,21,...,0,8,83,0,0,0,0,0,N/E,0
2020-05-25,"5,689",1,141,0,0,294,942,0,0,9,...,0,5,27,0,0,0,0,0,N/E,0
2020-05-26,"5,652",0,158,1,0,243,910,1,0,4,...,0,6,34,0,0,0,1,0,N/E,0
2020-05-27,"8,761",0,121,0,0,233,800,0,0,9,...,0,3,23,0,0,0,1,0,N/E,0
2020-05-28,"10,299",0,140,0,0,297,"1,003",0,0,15,...,0,8,31,0,0,0,2,0,N/E,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-26,"7,913",0,124,0,0,375,842,0,0,14,...,0,9,20,0,0,0,-1,0,N/E,0
2023-01-27,"8,602",0,124,0,1,449,857,0,0,26,...,0,15,30,0,0,2,0,0,N/E,0
2023-01-28,"6,835",0,78,1,0,391,888,0,0,19,...,0,12,17,-1,0,0,0,0,N/E,0


In [126]:
# Se guardó el data frame obtenido en un archivo .csv para un uso posterior. 
df_cuentas_validas.to_csv('/content/drive/MyDrive/Proyecto Python/Datasets/api_cuentas_validas.csv')